# Webpage to ICS Calendar file Generator.

In [ ]:
#Let's import the necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from ics import Calendar, Event
from datetime import datetime, timedelta
import os

Let's get the calendar webpage and parse it with beautiful soup.

In [ ]:

# URL of the webpage
url = input('Enter the URL of the BBC Proms calendar: ')

# Fetch the webpage content
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

Getting the BBC Proms Calendar webpage we can extract all the events into the concerts array

In [ ]:
# Extract concert details
concerts = []
for date_section in soup.find_all('h3', class_='ev-event-calendar__date'):
    date = date_section.get_text(strip=True)
    event_list = date_section.find_next('ul')
    if event_list:
        for event in event_list.find_all('li'):
            info_div = event.find('div', class_='ev-event-calendar__information')
            if info_div:
                name_div = info_div.find('div', class_='ev-event-calendar__name')
                time_div = info_div.find('div', class_='ev-event-calendar__time')
                description_tag = info_div.find('p', class_='ev-event-calendar__event-description')

                name = name_div.find('a').get_text(strip=True) if name_div else 'N/A'
                start_time = time_div.get_text(strip=True) if time_div else 'N/A'
                description = description_tag.get_text(strip=True) if description_tag else 'N/A'
                
                concerts.append({
                    'Date': date,
                    'Name': name,
                    'Start Time': start_time,
                    'Description': description
                })

# Convert to DataFrame
concerts_df = pd.DataFrame(concerts)

Once we've extracted the concerts infomation from the webpage now we can put this infomation into a calendar.

In [ ]:
# Create a new calendar
cal = Calendar()

# Convert DataFrame dates and times to datetime objects and create events
for index, row in concerts_df.iterrows():
    event = Event()
    event.name = row['Name']
    event.description = row['Description']
    
    # Handle date ranges and combine with start time
    date_str = row['Date'].split()[0:3]  # Take only the first date part
    date_str = " ".join(date_str)
    start_time_str = row['Start Time']
    
    try:
        event_start = datetime.strptime(f"{date_str} {start_time_str}", "%a %d %b %Y %H:%M")
    except ValueError:
        # Handle cases where time might not be available or incorrect format
        try:
            event_start = datetime.strptime(date_str, "%a %d %b %Y")
        except ValueError:
            continue
    
    event.begin = event_start
    event.end = event_start + timedelta(hours=2)  # assuming 2-hour events
    
    cal.events.add(event)

# Save the calendar to a file
# Get the directory of the current python file
current_dir = os.path.dirname(os.path.abspath(__file__))

# Set the path for the ical file in the same directory
ical_file_path = os.path.join(current_dir, 'bbc_proms_2024.ics')
with open(ical_file_path, 'w') as f:
    f.writelines(cal)

print(f'iCalendar file saved as {ical_file_path}')
